In [47]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit


file = open("../log/ (1).pickle", "rb")
data = pickle.load(file)
file.close()

# print(data)

In [48]:
game_info = data['ml_1P']['scene_info']
game_info2 = data['ml_2P']['scene_info']

game_command = data['ml_1P']['command']
game_command2 = data['ml_2P']['command']

for i in range(2, 42):
    path = "../log/ ("+str(i)+").pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    game_info = game_info + data['ml_1P']['scene_info']
    game_command = game_command + data['ml_1P']['command']
    file.close()
    
for i in range(2, 42):
    path = "../log/ ("+str(i)+").pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    game_info2 = game_info2 + data['ml_2P']['scene_info']
    game_command2 = game_command2 + data['ml_2P']['command']
    file.close()
    
print(len(game_info))
print(len(game_command))

print(len(game_info2))
print(len(game_command2))

30090
30090
30090
30090


In [49]:

g = game_info[1]
g2 = game_info2[1]

feature = np.array([g['ball'][0], g['ball'][1], g['platform_1P'][0]])
feature2 = np.array([g['ball'][0], g['ball'][1], g['platform_2P'][0]])

game_command[1] = 0
game_command2[1] = 0

print(len(game_info))
print(len(game_info2))

for i in range(2, len(game_info) - 1):
    g = game_info[i]
    feature = np.vstack((feature, [g['ball'][0], g['ball'][1], g['platform_1P'][0]]))
    if game_command[i] == "NONE": game_command[i] = 0
    elif game_command[i] == "MOVE_LEFT": game_command[i] = 1
    else: game_command[i] = 2
        
for i in range(2, len(game_info2) - 1):
    g2 = game_info2[i]
    feature2 = np.vstack((feature2, [g2['ball'][0], g2['ball'][1], g2['platform_2P'][0]]))
    if game_command2[i] == "NONE": game_command2[i] = 0
    elif game_command2[i] == "MOVE_LEFT": game_command2[i] = 1
    else: game_command2[i] = 2
    
answer = np.array(game_command[1:-1])
answer2 = np.array(game_command2[1:-1])

print(feature)
print(feature2)

print(feature.shape)
print(feature2.shape)

print(answer)
print(answer2)

print(answer.shape)
print(answer2.shape)

30090
30090
[[ 98  80  80]
 [ 91  87  75]
 [ 84  94  70]
 ...
 [ 64 404 120]
 [ 80 420 115]
 [ 96 436 110]]
[[ 98  80  80]
 [ 91  87  85]
 [ 84  94  90]
 ...
 [ 64 404 140]
 [ 80 420 145]
 [ 96 436 150]]
(30088, 3)
(30088, 3)
[0 1 1 ... 1 1 1]
[0 2 2 ... 2 2 2]
(30088,)
(30088,)


In [55]:
#資料劃分
x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=1/3, random_state=25)
#參數區間
param_grid = {'n_neighbors':[1, 2, 3]}
#交叉驗證 
cv = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=12)
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=cv, verbose=10, n_jobs=-1) #n_jobs為平行運算的數量
grid.fit(x_train, y_train)
grid_predictions = grid.predict(x_test)

#儲存
file = open('model1.pickle', 'wb')
pickle.dump(grid, file)
file.close()

Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:    2.3s remaining:    4.6s
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    2.3s remaining:    2.3s
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    2.3s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    2.3s finished


In [56]:
#資料劃分
x_train, x_test, y_train, y_test = train_test_split(feature2, answer2, test_size=1/3, random_state=42)
#參數區間
param_grid = {'n_neighbors':[1, 2, 3]}
#交叉驗證 
cv = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=12)
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=cv, verbose=10, n_jobs=-1) #n_jobs為平行運算的數量
grid.fit(x_train, y_train)
grid_predictions = grid.predict(x_test)

#儲存
file = open('model2.pickle', 'wb')
pickle.dump(grid, file)
file.close()

Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:    0.2s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.3s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    1.1s finished


In [57]:
#最佳參數
print(grid.best_params_)
#預測結果
#print(grid_predictions)
#混淆矩陣
print(confusion_matrix(y_test, grid_predictions))
#分類結果
print(classification_report(y_test, grid_predictions))

{'n_neighbors': 1}
[[ 641  315  301]
 [ 315 3212  907]
 [ 326  935 3078]]
              precision    recall  f1-score   support

           0       0.50      0.51      0.50      1257
           1       0.72      0.72      0.72      4434
           2       0.72      0.71      0.71      4339

    accuracy                           0.69     10030
   macro avg       0.65      0.65      0.65     10030
weighted avg       0.69      0.69      0.69     10030

